In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')

train_x = train.drop(columns='label').values
train_y = train['label'].values
train_x = train_x.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

print(len(train_y), len(train_x))

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=1)

print(len(train_x), len(train_y), len(val_x), len(val_y))

train_x = train_x.astype('float32')
val_x = val_x.astype('float32')
test = test.astype('float32')

train_x /= 255
val_x /= 255
test /= 255

print(len(train_y), len(train_x))

plt.imshow(train_x[0], cmap='binary')

In [ ]:
model = Sequential()

model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu", input_shape=(28,28,1)))
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())    
model.add(Conv2D(filters=256, kernel_size = (3,3), activation="relu"))
    
model.add(MaxPooling2D(pool_size=(2,2)))
    
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(512,activation="relu"))
    
model.add(Dense(10,activation="softmax"))
    
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# train_y size not correct since the output is 5 neuron, so chage it to an array
train_y_aug = []
val_y_aug = []
temp = [0 for i in range(10)]

for j in train_y:
    temp = [0 for i in range(10)]
    temp[j] = 1
    train_y_aug.append(temp)
    
for a in val_y:
    temp = [0 for i in range(10)]
    temp[j] = 1
    val_y_aug.append(temp)
    
train_y_aug = np.asarray(train_y_aug)

In [ ]:
model.fit(train_x, train_y_aug, epochs=20)

In [ ]:
pred = model.predict(val_x)
real_pred = []

for i in range(len(pred)):
    real_pred.append(np.argmax(pred[i]))

val_acc = accuracy_score(real_pred, val_y)
print("accuracy on validation set is:", val_acc*100, "%")

In [ ]:
test_pred = model.predict(test)
real_test_pred = []

for i in range(len(test_pred)):
    real_test_pred.append(np.argmax(test_pred[i]))
    
# print(real_test_pred)

sub = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
sub['Label'] = real_test_pred

sub.to_csv('submission.csv', index=False)

sub.head()

In [ ]:
#datagen = ImageDataGenerator(
#       featurewise_center=False,  # set input mean to 0 over the dataset
#       samplewise_center=False,  # set each sample mean to 0
#       featurewise_std_normalization=False,  # divide inputs by std of the dataset
#       samplewise_std_normalization=False,  # divide each input by its std
#       zca_whitening=False,  # apply ZCA whitening
#       rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
#       zoom_range = 0.1, # Randomly zoom image 
#       width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#       height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#       horizontal_flip=False,  # randomly flip images
#       vertical_flip=False)  # randomly flip images

#train_x_gen = data_gen.flow(train_x, train_y, batch_size=32)
#val_x_gen = data_gen.flow(val_x, val_y, batch_size=32)

In [ ]:

#history = model.fit_generator(train_x_gen, 
#                             epochs = 20, 
#                             steps_per_epoch = train_x.shape[0] // 32,
#                             validation_data = val_x_gen,
#                             validation_steps = val_x.shape[0] // 32)